In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import StandardScaler

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from model import DenseNet, Model, DenseBlock

from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2

data_dir = './DATA/lish-moa/'
SEED = 123
np.random.seed(SEED)
torch.manual_seed(SEED)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load data set

In [15]:
X = pd.read_csv(data_dir+'train_features.csv', index_col='sig_id')
y = pd.read_csv(data_dir+'train_targets_scored.csv', index_col='sig_id')

## Preprocess

In [16]:
# One-hot encoding for cp_type and cp_dose
X['cp_type'].replace({'trt_cp':1., 'ctl_vehicle':0.}, inplace=True)
X['cp_dose'].replace({'D1':1., 'D2':0.}, inplace=True)

# split into training set and test set
ids = X.index.values.copy()
np.random.shuffle(ids)

train_perc, test_perc = 0.8, 0.2
train_id = ids[:round(len(ids)*train_perc)]
test_id = ids[round(len(ids)*train_perc):]

X_train = X.loc[train_id]
X_test = X.loc[test_id]

y_train = y.loc[train_id]
y_test = y.loc[test_id]

# normalize the data
scaler = StandardScaler()
X_train_norm = pd.DataFrame(scaler.fit_transform(X_train))
X_train_norm.columns = X_train.columns
X_train_norm.index = X_train.index

X_test_norm = pd.DataFrame(scaler.transform(X_test))
X_test_norm.columns = X_test.columns
X_test_norm.index = X_test.index

## Model

In [17]:
params_net = {'input_size': X_train_norm.shape[1],
             'hidden_size': [1024, 512, 1024],
             'output_size': y_train.shape[1],
             'dropout': 0.01}

params_fit = {'X':X_train_norm,
             'y': y_train,
             'epoch': 5,
             'lr': 1e-4,
             'batch_size':128,
             'L1': 1e-6,
             'L2': 1e-6,
             'pos_weight':1,
             'verbose':True}

net = DenseNet(**params_net)
model = Model(net)
model.fit(**params_fit)

Epoch [1, 0] : loss 1.0131256580352783
Epoch [1, 3840] : loss 0.9746132493019104
Epoch [1, 7680] : loss 0.8820200562477112
Epoch [1, 11520] : loss 0.8355883955955505
Epoch [1, 15360] : loss 0.8230900764465332
Epoch [2, 0] : loss 0.8132508993148804
Epoch [2, 3840] : loss 0.799078643321991
Epoch [2, 7680] : loss 0.7966324687004089
Epoch [2, 11520] : loss 0.7878764867782593
Epoch [2, 15360] : loss 0.778723955154419
Epoch [3, 0] : loss 0.7944122552871704
Epoch [3, 3840] : loss 0.7877381443977356
Epoch [3, 7680] : loss 0.7827826738357544
Epoch [3, 11520] : loss 0.7662061452865601
Epoch [3, 15360] : loss 0.7677496075630188
Epoch [4, 0] : loss 0.767212450504303
Epoch [4, 3840] : loss 0.7721747756004333
Epoch [4, 7680] : loss 0.7701340913772583
Epoch [4, 11520] : loss 0.7616013884544373
Epoch [4, 15360] : loss 0.7590694427490234
Epoch [5, 0] : loss 0.7623365521430969
Epoch [5, 3840] : loss 0.763582706451416
Epoch [5, 7680] : loss 0.7567314505577087
Epoch [5, 11520] : loss 0.7538787126541138
Ep

In [ ]:
y_train_pred = model.predict_proba(X_train_norm)
y_test_pred = model.predict_proba(X_test_norm)

In [10]:
net = DenseBlock(3, 2, 0.02, activate=F.relu)

In [12]:
a = torch.rand(5, 3)
a

tensor([[0.1841, 0.7264, 0.3153],
        [0.6871, 0.0756, 0.1966],
        [0.3164, 0.4017, 0.1186],
        [0.8274, 0.3821, 0.6605],
        [0.8536, 0.5932, 0.6367]])

In [13]:
net(a)

tensor([[0.0318, 0.0000],
        [0.0000, 0.0795],
        [0.1438, 0.0000],
        [0.0000, 0.0000],
        [0.0000, 0.0000]], grad_fn=<ReluBackward0>)

In [3]:
for a in net.linear.parameters():
    print(a)

Parameter containing:
tensor([[-0.2354,  0.0191, -0.2867],
        [ 0.2177, -0.4919,  0.4232]], requires_grad=True)
Parameter containing:
tensor([-0.4196, -0.4590], requires_grad=True)


In [14]:
torch.cat([x.view(-1) for x in net.linear.parameters()])

tensor([-0.4545, -0.4456, -0.5394,  0.2368,  0.1636, -0.5732, -0.2033, -0.5588],
       grad_fn=<CatBackward>)

In [5]:
list(net.linear.parameters())[0].view(-1)

tensor([-0.2354,  0.0191, -0.2867,  0.2177, -0.4919,  0.4232],
       grad_fn=<ViewBackward>)

In [6]:
a = torch.tensor([])
a = torch.cat([a, list(net.linear.parameters())[0].view(-1)])

In [7]:
a

tensor([-0.2354,  0.0191, -0.2867,  0.2177, -0.4919,  0.4232],
       grad_fn=<CatBackward>)

In [8]:
b = torch.cat([a, list(net.linear.parameters())[0].view(-1)])
b

tensor([-0.2354,  0.0191, -0.2867,  0.2177, -0.4919,  0.4232, -0.2354,  0.0191,
        -0.2867,  0.2177, -0.4919,  0.4232], grad_fn=<CatBackward>)

In [23]:
net2 = DenseNet(**params_net)
net2.regular_loss(0.001, 0.001)

tensor(38.3275, grad_fn=<AddBackward0>)